In [64]:
from SE3Lie import *
import numpy as np
from sympy import *
from pyecca.lie import so3, r3, se3
from pyecca.lie.util import DirectProduct
from pyecca.lie.so3 import Quat, Dcm, Euler, Mrp
from pyecca.lie.r3 import R3
from pyecca.lie.se3 import SE3
from pyecca.test import test_lie
import casadi

In [57]:
x,y,z,i,j,k = symbols('x,y,z,i,j,k', real = True)

In [58]:
input = Matrix([x,y,z,i,j,k])
input

Matrix([
[x],
[y],
[z],
[i],
[j],
[k]])

In [59]:
    r = ca.DM([0.1, 0.2, 0.3])
    v = ca.DM([0.1, 0.2, 0.3])
    R = Dcm.from_euler(ca.DM([0.1, 0.2, 0.3]))

In [60]:
inp = ca.horzcat(np.transpose(r),np.transpose(v))

In [61]:
veeSe3 = np.transpose(inp)
veeSe3

array([[0.1],
       [0.2],
       [0.3],
       [0.1],
       [0.2],
       [0.3]])

In [62]:
eps = 1e-7 # to avoid divide by zero
x = ca.SX.sym('x')
C1 = ca.Function('a', [x], [ca.if_else(ca.fabs(x) < eps, 1 - x ** 2 / 6 + x ** 4 / 120, ca.sin(x)/x)])
C2 = ca.Function('b', [x], [ca.if_else(ca.fabs(x) < eps, 0.5 - x ** 2 / 24 + x ** 4 / 720, (1 - ca.cos(x)) / x ** 2)])
del x

In [51]:
SE3_diff_corr.se3_diff_correction_inv(veeSe3,C1,C2)

TypeError: unsupported operand type(s) for *: 'Function' and 'float'

In [54]:
C1

Function(a:(i0)->(o0) SXFunction)

In [55]:
C2

Function(b:(i0)->(o0) SXFunction)

In [66]:
se3LieAlg(veeSe3,C1,C2).adjoint_se3()

NameError: name 'se3LieAlg' is not defined

In [69]:
adjoint_se3(veeSe3)

NameError: name 'adjoint_se3' is not defined

NameError: name 'test_so3' is not defined